In [51]:
from darkflow.net.build import TFNet
import cv2
import numpy as np
from PIL import Image
import glob

options = {"model": "cfg/yolo.cfg", "load": "bin/yolo.weights", "threshold": 0.1, "gpu": 1}

tfnet = TFNet(options)

class_names = ['aeroplane', 'bicycle', 'bird', 'boat', 'bottle',
              'bus', 'car', 'cat', 'chair', 'cow', 'diningtable',
              'dog', 'horse', 'motorbike', 'person', 'pottedplant',
              'sheep', 'sofa', 'train', 'tvmonitor']

files = glob.glob("./sample_img/*")
for f in files:
    imgcv = cv2.imread(f)
# imgcv = cv2.imread("./sample_img/386_720.jpeg")
    result = tfnet.return_predict(imgcv)
#     print(result)

    num_classes = len(class_names)
    class_colors = []
    for i in range(0, num_classes):
        hue = 255*i/num_classes
        col = np.zeros((1,1,3)).astype("uint8")
        col[0][0][0] = hue
        col[0][0][1] = 128
        col[0][0][2] = 255
        cvcol = cv2.cvtColor(col, cv2.COLOR_HSV2BGR)
        col = (int(cvcol[0][0][0]), int(cvcol[0][0][1]), int(cvcol[0][0][2]))
        class_colors.append(col)

    for item in result:
        tlx = item['topleft']['x']
        tly = item['topleft']['y']
        brx = item['bottomright']['x']
        bry = item['bottomright']['y']
        label = item['label']
        conf = item['confidence']
        if item['label'] == 'person':
            tl_x = tlx
            tl_y = tly
            br_x = brx
            br_y = bry
    
        if conf > 0.6:

            for i in class_names:
                if label == i:
                    class_num = class_names.index(i)
                    break

            #枠の作成
            cv2.rectangle(imgcv, (tlx, tly), (brx, 666), class_colors[class_num], 2)

            #ラベルの作成
#             text = label + " " + ('%.2f' % conf)
#             cv2.rectangle(imgcv, (tlx, tly - 15), (tlx + 100, tly + 5), class_colors[class_num], -1)
#             cv2.putText(imgcv, text, (tlx, tly), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1)

# 書き出し
    im = Image.open(f)
    im_crop = im.crop((tl_x, tl_y, br_x, 666))
    im_crop.save(f, quality=95)

Parsing ./cfg/yolo.cfg
Parsing cfg/yolo.cfg
Loading bin/yolo.weights ...
Successfully identified 203934260 bytes
Finished in 0.03419661521911621s
Model has a coco model name, loading coco labels.

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
       |        | input                            | (?, 608, 608, 3)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 608, 608, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 304, 304, 32)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 304, 304, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 76, 76, 128)
 Load  |  Yep!

In [76]:
# Specify the paths for the 2 files
protoFile = "pose_deploy_linevec_faster_4_stages.prototxt"
weightsFile = "pose_iter_160000.caffemodel"
 
# Read the network into Memory
net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)

import time
MODE = "MPI"

if MODE is "COCO":
    protoFile = "pose/coco/pose_deploy_linevec.prototxt"
    weightsFile = "pose/coco/pose_iter_440000.caffemodel"
    nPoints = 18
    POSE_PAIRS = [ [1,0],[1,2],[1,5],[2,3],[3,4],[5,6],[6,7],[1,8],[8,9],[9,10],[1,11],[11,12],[12,13],[0,14],[0,15],[14,16],[15,17]]

elif MODE is "MPI" :
    protoFile = "pose_deploy_linevec_faster_4_stages.prototxt"
    weightsFile = "pose_iter_160000.caffemodel"
    nPoints = 15
    POSE_PAIRS = [[0,1], [1,2], [2,3], [3,4], [1,5], [5,6], [6,7], [1,14], [14,8], [8,9], [9,10], [14,11], [11,12], [12,13] ]

files = glob.glob("./sample_img/*")
for f in files:
    frame = cv2.imread(f)
    frameCopy = np.copy(frame)
    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]
    threshold = 0.1

    net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)
    t = time.time()
    # input image dimensions for the network
    inWidth = 368
    #368
    inHeight = 368
    #368
    inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight),
                              (0, 0, 0), swapRB=False, crop=False)

    net.setInput(inpBlob)

    output = net.forward()
    print("time taken by network : {:.3f}".format(time.time() - t))

    H = output.shape[2]
    W = output.shape[3]

    # Empty list to store the detected keypoints
    points = []
    for i in range(nPoints):
        # confidence map of corresponding body's part.
        probMap = output[0, i, :, :]

        # Find global maxima of the probMap.
        minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

        # Scale the point to fit on the original image
        x = (frameWidth * point[0]) / W
        y = (frameHeight * point[1]) / H

        if prob > threshold : 
            cv2.circle(frameCopy, (int(x), int(y)), 8, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
            cv2.putText(frameCopy, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, lineType=cv2.LINE_AA)

            # Add the point to the list if the probability is greater than the threshold
            points.append((int(x), int(y)))
        else :
            points.append(None)
    # Draw Skeleton
    for pair in POSE_PAIRS:
        partA = pair[0]
        partB = pair[1]

        if points[partA] and points[partB]:
            cv2.line(frame, points[partA], points[partB], (0, 255, 255), 2)
            cv2.circle(frame, points[partA], 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)

#     cv2.imwrite('./sample_img/Output-Keypoints.jpg', frameCopy)
#     cv2.imwrite('./sample_img/Output-Skeleton.jpg', frame)

    print("Total time taken : {:.3f}".format(time.time() - t))

    cv2.waitKey(0)

    x=[]
    y=[]
    if points[8] is not None and points[11] is not None:
        eight_y = points[8][1]
        eleven_y = points[11][1]
        if eight_y > eleven_y :
            waist_y = eleven_y
        else:
            waist_y = eight_y
    while None in points:
        points.remove(None)
    print(points)
    if len(points) !=0:
        for i in range(len(points)):
            x.append(points[i][0])
            y.append(points[i][1])
        
        min_x=min(sorted(x))
        max_x=max(sorted(x))
        second_min_y=sorted(y)[2]
        max_y=max(sorted(y))
        print(min_x,max_x,second_min_y,max_y)
        img = cv2.imread(f)
        img_1 = cv2.rectangle(img,(min_x-20,second_min_y-20),(max_x+20,waist_y),(0,255,0),3)
        img_2 = cv2.rectangle(img_1,(min_x,waist_y+3),(max_x,max_y),(255,0,0),3)
        cv2.imwrite(f,img_2)

time taken by network : 0.736
Total time taken : 0.737
[(195, 14), (228, 87), (163, 145), (228, 217), (195, 145), (293, 87), (315, 203), (260, 145), (293, 319), (315, 464), (391, 522), (336, 290), (402, 420), (445, 522), (282, 217)]
163 445 87 522
time taken by network : 0.718
Total time taken : 0.719
[(228, 29), (380, 174), (391, 217), (228, 29), (195, 174), (271, 261), (228, 246), (271, 145)]
195 391 145 261
time taken by network : 0.734
Total time taken : 0.735
[(239, 29), (239, 116), (173, 145), (141, 246), (195, 174), (293, 145), (304, 246), (304, 333), (206, 348), (184, 478), (239, 580), (271, 362), (250, 536), (173, 594), (239, 246)]
141 304 145 594
time taken by network : 0.723
Total time taken : 0.724
[(184, 43), (206, 116), (260, 116), (260, 304), (260, 304), (173, 174), (184, 261), (206, 319), (239, 348), (260, 435), (315, 536), (195, 348), (217, 449), (239, 565), (217, 261)]
173 315 116 565
time taken by network : 0.725
Total time taken : 0.726
[(239, 304), (369, 188), (369

In [39]:
print()

./sample_img/detected


In [62]:
points[11] is not None

True

In [46]:
if not points:
    print(a)

./sample_img/360_1925.jpeg
